In [ ]:
get_ipython().run_cell_magic('capture', '', '!pip install db-dtypes\n!pip install keras\n!pip install tensorflow\n')

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\n')

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query_main = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
query_job = bigquery_client.query(query_main)
unemployment_data = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()

In [ ]:
query_pred = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
query_job = bigquery_client.query(query_pred)
prediction_list = query_job.to_dataframe()

In [ ]:
print(unemployment_data.info())
print(wage_data.info())

In [ ]:
# check shape of both frames to see if they are joinable
print('Unemployment df size:', unemployment_data.shape)
print('Wage df size:', wage_data.shape)

In [ ]:
unemployment_data.isnull().sum() / len(unemployment_data) * 100

In [ ]:
wage_data.isnull().sum() / len(wage_data) * 100

In [ ]:
# replace values with 0
clean_unemploymentDf = unemployment_data.copy()
clean_unemploymentDf.fillna(0, inplace=True)
clean_unemploymentDf

In [ ]:
clean_unemploymentDf.isnull().sum() #check

In [ ]:
get_ipython().run_cell_magic('capture', '', "unemp_dupl = clean_unemploymentDf[clean_unemploymentDf.duplicated()]\nprint('Duplicate rows: ', unemp_dupl)\n# ignore duplicates - different time periods\n")

In [ ]:
# check correlation
correlation = clean_unemploymentDf.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f')

In [ ]:
# check wage information
wage_data[wage_data['average_wage'].isnull()] # there's 3 nulls here - might as well drop them and use this tract to attempt to join the datasets together; or impute with mean

In [ ]:
#wage_data.dropna(axis=0, inplace=True)
wage_data['average_wage'].fillna(wage_data['average_wage'].mean(), inplace=True)
wage_data.isnull().sum()

In [ ]:
wage_dupl = wage_data[wage_data.duplicated()]
print('Duplicate rows: ', wage_dupl)

In [ ]:
# join df on tract
main_df = pd.merge(clean_unemploymentDf, wage_data, on=['tract', 'uu_id'], how='outer')

In [ ]:
main_df

In [ ]:
# drop columns created by merge and rename existing columns to original
main_df.drop(['countyfips_y','tract_name_y'], axis=1, inplace=True)
main_df.rename({'countyfips_x':'countyfips', 'tract_name_x':'tract_name'}, axis=1, inplace=True)

In [ ]:
main_df.isnull().sum() # check again

In [ ]:
main_df.dropna(axis=0, inplace=True) #most of the rows with NA values have nothing to add and cannot be imputed

In [ ]:
# check correlation again for new, merged frame
correlation = main_df.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f')

In [ ]:
main_df.isnull().sum()

In [ ]:
def evaluate_regressor(prediction_dataframe):
    # Takes in a prediction dataframe of 2 columns, Actual values and Predicted values generated by a regressor
    # Outputs MSE, MAR, RMSE and MAPE metrics. Must have columns named Actual and Predicted.
    print('MSE:', mean_squared_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted']))
    print('MAE:', mean_absolute_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted']))
    print('RMSE:', np.sqrt(mean_squared_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted'])))
    print('MAPE:', np.mean(np.abs((prediction_dataframe['Actual'] - prediction_dataframe['Predicted']) / prediction_dataframe['Actual'])) * 100)

In [ ]:
def get_predictions(regressor, model_type, name):
    # generates predictions for any model and writes out a dataframe in csv containing them
    # takes a regressor and learning method type as input: DL and ML
    # DL/ML variable basically changes the shape for an input from a 2D array to 3D arry, as required tensor shape
    result_list = []
    uu_id_transform = LE.fit_transform(prediction_list['uu_id'])
    if model_type == 'DL':
        predict_arr = np.array(SC_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, 43, axis=1)
            to_predict = np.reshape(to_predict, (to_predict.shape[0], to_predict.shape[1],1))
            r = regressor.predict(to_predict)
            result_list.append(r)
        result_list = np.array(result_list)
        result_list = np.reshape(result_list, (619,))
    elif model_type == 'ML':
        predict_arr = np.array(RB_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, 43, axis=1)
            r = regressor.predict(to_predict)
            result_list.append(r)
    result_df = pd.DataFrame(result_list, columns = ['Predictions'])
    prediction_sub = prediction_list.copy()
    prediction_sub['total_claims'] = result_df.values
    prediction_sub = prediction_sub[['uu_id','total_claims','week_number']]
    os.makedirs('lost+found/submission_files', exist_ok=True)
    prediction_sub.to_csv('lost+found/submission_files/'+name+'.csv', index=False)
    return prediction_sub

In [ ]:
def get_pred_frame(test_frame, prediction_array):
    prediction_frame = pd.DataFrame({'Actual': test_frame, 'Predicted': prediction_array.flatten()})
    return prediction_frame

In [ ]:
# updated_ingest = pd.concat([merged_ingest, combined_ingest])
ingest = pd.read_csv('lost+found/submission_files/updated_ingest.csv')

In [ ]:
ingest

In [ ]:
ingest.shape

In [ ]:
new_data = main_df.copy()
complete_ingest = pd.concat([ingest, new_data])
complete_ingest.shape

In [ ]:
print(complete_ingest.shape)
print(ingest.shape)

In [ ]:
complete_ingest.isnull().sum()

In [ ]:
complete_ingest.dropna(axis=0, inplace=True)

In [ ]:
complete_ingest.shape

In [ ]:
# check if new data is duplicate
complete_ingest.equals(main_df)

In [ ]:
## write out combined data for use later
os.makedirs('lost+found/submission_files', exist_ok=True)
complete_ingest.to_csv('lost+found/submission_files/complete_ingest.csv', index=False)

In [ ]:
prediction_list.to_csv('prediction_list.csv')

In [ ]:
get_ipython().run_cell_magic('capture', '', '!pip install db-dtypes\n!pip install keras\n!pip install tensorflow\n')

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\n')

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query_main = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
query_job = bigquery_client.query(query_main)
unemployment_data = query_job.to_dataframe()

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()

In [ ]:
query_pred = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
query_job = bigquery_client.query(query_pred)
prediction_list = query_job.to_dataframe()

In [ ]:
# replace values with 0
clean_unemploymentDf = unemployment_data.copy()
clean_unemploymentDf.fillna(0, inplace=True)
clean_unemploymentDf

In [ ]:
clean_unemploymentDf.isnull().sum() #check

In [ ]:
# check correlation
correlation = clean_unemploymentDf.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f')

In [ ]:
# check wage information
wage_data[wage_data['average_wage'].isnull()] # there's 3 nulls here - might as well drop them and use this tract to attempt to join the datasets together; or impute with mean

In [ ]:
#wage_data.dropna(axis=0, inplace=True)
wage_data['average_wage'].fillna(wage_data['average_wage'].mean(), inplace=True)
wage_data.isnull().sum()

In [ ]:
wage_dupl = wage_data[wage_data.duplicated()]
print('Duplicate rows: ', wage_dupl)

In [ ]:
# join df on tract
main_df = pd.merge(clean_unemploymentDf, wage_data, on=['tract', 'uu_id'], how='outer')

In [ ]:
main_df

In [ ]:
# drop columns created by merge and rename existing columns to original
main_df.drop(['countyfips_y','tract_name_y'], axis=1, inplace=True)
main_df.rename({'countyfips_x':'countyfips', 'tract_name_x':'tract_name'}, axis=1, inplace=True)

In [ ]:
main_df.isnull().sum() # check again

In [ ]:
main_df.dropna(axis=0, inplace=True) #most of the rows with NA values have nothing to add and cannot be imputed

In [ ]:
# check correlation again for new, merged frame
correlation = main_df.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
plt.figure(figsize=(15,10))
sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f')

In [ ]:
main_df.isnull().sum()

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\nfrom keras.models import load_model\n')

In [ ]:
# load model
StackLSTM_Regressor = load_model('BiDLSTM_v1-04.h5')

In [ ]:
# summary for viewers
StackLSTM_Regressor.summary()

In [ ]:
# quick preprocess to keep uu_id and scale values
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
LE = LabelEncoder()
RB_other = RobustScaler()
SC_other = StandardScaler()
# RB_claims = RobustScaler()

In [ ]:
# this needs different feature engineering, so I'm starting from scratch
DL_data = updated_ingest.copy()

In [ ]:
# this needs different feature engineering, so I'm starting from scratch
DL_data = complete_ingest.copy()

In [ ]:
drop_these = ['timeperiod','tract','top_category_employer1','top_category_employer2','top_category_employer3','tract_name','countyfips', 'edu_unknown', 'gender_na', 'race_noanswer']
DL_data['uu_id'] = LE.fit_transform(DL_data['uu_id'])
scale_these = ['edu_8th_or_less', 'edu_grades_9_11', 'edu_hs_grad_equiv', 'edu_post_hs', 'gender_female', 'gender_male', 
            'race_amerindian', 'race_asian', 'race_black', 'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
DL_data.drop(drop_these, axis = 1, inplace=True)
DL_data[scale_these] = SC_other.fit_transform(DL_data[scale_these])

In [ ]:
DL_data.columns

In [ ]:
# split set
DL_Y = DL_data['total_claims']
DL_X = DL_data[['uu_id', 'week_number', 'edu_8th_or_less', 'edu_grades_9_11', 'edu_hs_grad_equiv', 'edu_post_hs', 'gender_female', 'gender_male',
               'race_amerindian', 'race_asian', 'race_black', 'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']]
DL_XTrain, DL_XTest, DL_YTrain, DL_YTest = train_test_split(DL_X, DL_Y, test_size=0.20, random_state=69)

In [ ]:
# change to np vectors
DL_XTrain = DL_XTrain.to_numpy()
DL_XTest = DL_XTest.to_numpy()

In [ ]:
# reshape because F*** tensors
DL_XTrain = np.reshape(DL_XTrain, (DL_XTrain.shape[0], DL_XTrain.shape[1], 1))

In [ ]:
# convert X and Y train to float because input dtype accepts floats only
DL_YTrain = DL_YTrain.astype(float)
DL_XTrain = DL_XTrain.astype(float)

In [ ]:
# float cast
DL_XTest = DL_XTest.astype(float)
# make predictions
DL_XTest = np.reshape(DL_XTest, (DL_XTest.shape[0], DL_XTest.shape[1],1))
predictions = StackLSTM_Regressor.predict(DL_XTest)

In [ ]:
get_pred_frame(DL_YTest, predictions)

In [ ]:
evaluate_regressor(get_pred_frame(DL_YTest, predictions))

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(DL_YTest.values, color = 'red', label = 'Actual values')
plt.plot(predictions, color='blue', label='Predicted values')
plt.title('Model Prediction Visual')
plt.legend()
plt.show()

In [ ]:
get_predictions(StackLSTM_Regressor, 'DL', 'submission_prediction_output')

In [ ]:
def get_predictions(regressor, model_type, name):
    # generates predictions for any model and writes out a dataframe in csv containing them
    # takes a regressor and learning method type as input: DL and ML
    # DL/ML variable basically changes the shape for an input from a 2D array to 3D arry, as required tensor shape
    result_list = []
    uu_id_transform = LE.fit_transform(prediction_list['uu_id'])
    if model_type == 'DL':
        predict_arr = np.array(SC_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, 43, axis=1)
            to_predict = np.reshape(to_predict, (to_predict.shape[0], to_predict.shape[1],1))
            r = regressor.predict(to_predict)
            result_list.append(r)
        result_list = np.array(result_list)
        result_list = np.reshape(result_list, (525,))
    elif model_type == 'ML':
        predict_arr = np.array(RB_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, 43, axis=1)
            r = regressor.predict(to_predict)
            result_list.append(r)
    result_df = pd.DataFrame(result_list, columns = ['Predictions'])
    prediction_sub = prediction_list.copy()
    prediction_sub['total_claims'] = result_df.values
    prediction_sub = prediction_sub[['uu_id','total_claims','week_number']]
    os.makedirs('lost+found/submission_files', exist_ok=True)
    prediction_sub.to_csv('lost+found/submission_files/'+name+'.csv', index=False)
    return prediction_sub

In [ ]:
get_predictions(StackLSTM_Regressor, 'DL', 'submission_prediction_output')

In [ ]:
def get_predictions(regressor, model_type, name, week):
    # generates predictions for any model and writes out a dataframe in csv containing them
    # takes a regressor and learning method type as input: DL and ML
    # DL/ML variable basically changes the shape for an input from a 2D array to 3D arry, as required tensor shape
    result_list = []
    uu_id_transform = LE.fit_transform(prediction_list['uu_id'])
    if model_type == 'DL':
        predict_arr = np.array(SC_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, week, axis=1)
            to_predict = np.reshape(to_predict, (to_predict.shape[0], to_predict.shape[1],1))
            r = regressor.predict(to_predict)
            result_list.append(r)
        result_list = np.array(result_list)
        result_list = np.reshape(result_list, (525,))
    elif model_type == 'ML':
        predict_arr = np.array(RB_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, week, axis=1)
            r = regressor.predict(to_predict)
            result_list.append(r)
    result_df = pd.DataFrame(result_list, columns = ['Predictions'])
    prediction_sub = prediction_list.copy()
    prediction_sub['total_claims'] = result_df.values
    prediction_sub = prediction_sub[['uu_id','total_claims','week_number']]
    os.makedirs('lost+found/submission_files', exist_ok=True)
    prediction_sub.to_csv('lost+found/submission_files/'+name+'.csv', index=False)
    return prediction_sub

In [ ]:
get_predictions(StackLSTM_Regressor, 'DL', 'submission_prediction_output', 43)

In [ ]:
prediction_list

In [ ]:
get_predictions(StackLSTM_Regressor, 'DL', 'submission_prediction_output', 44)

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\nfrom keras.models import load_model\nimport joblib\nfrom joblib import Parallel, delayed\n')

In [ ]:
# updated_ingest = pd.concat([merged_ingest, combined_ingest])
ingest = pd.read_csv('lost+found/submission_files/completed_ingest.csv')

In [ ]:
# updated_ingest = pd.concat([merged_ingest, combined_ingest])
ingest = pd.read_csv('lost+found/submission_files/complete_ingest.csv')

In [ ]:
# quick preprocess to keep uu_id and scale values
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
LE = LabelEncoder()
RB_other = RobustScaler()
SC_other = StandardScaler()
# RB_claims = RobustScaler()

In [ ]:
# set target and preprocess again
to_drop = ['timeperiod','tract','total_claims','top_category_employer1','top_category_employer2','top_category_employer3','tract_name','countyfips', 'edu_unknown', 'gender_na', 'race_noanswer']
Y = ingest['total_claims']
X = ingest.drop(to_drop, axis=1)
X['uu_id'] = LE.fit_transform(X['uu_id'])
to_scale = ['edu_8th_or_less', 'edu_grades_9_11', 'edu_hs_grad_equiv', 'edu_post_hs', 'gender_female', 'gender_male', 
            'race_amerindian', 'race_asian', 'race_black', 'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
X[to_scale] = RB_other.fit_transform(X[to_scale])
#X[['total_claims']] = RB_claims.fit_transform(main_df[['total_claims']])

In [ ]:
# import
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20, random_state=69)

In [ ]:
# load model
RFR_Regressor = joblib.load('RF_v1-5.pkl')

In [ ]:
Y_pred_RFR = RFR_Regressor.predict(X_test.values).reshape(-1,1)

In [ ]:
# evaluate
evaluate_regressor(get_pred_frame(Y_test,Y_pred_RFR))

In [ ]:
# call func
get_predictions(RFR_Regressor, 'ML', 'submission_prediction_output_RFR', 44)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(Y_Test.values, color = 'red', label = 'Actual values')
plt.plot(Y_pred_RFR, color='blue', label='Predicted values')
plt.title('Model Prediction Visual')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(Y_test.values, color = 'red', label = 'Actual values')
plt.plot(Y_pred_RFR, color='blue', label='Predicted values')
plt.title('Model Prediction Visual')
plt.legend()
plt.show()